In [ ]:
import os
from coffea.nanoevents import NanoEventsFactory
from coffea import processor

import dask_awkward as dak
import hist.dask as hda

import warnings
warnings.filterwarnings("error", module="coffea.*")


use_taskvine = True

if use_taskvine:
    from ndcctools.taskvine import DaskVine

    
    vine_manager = DaskVine(name=f"{os.environ["USER"]}-coffea-example-4")

    executor_args = {
        "scheduler": vine_manager,
        "worker_transfers": True,
        # "task_mode": "function-calls",
    }
else:
    from distributed import Client
    client = Client()

    executor_args = {}

In [ ]:
data_abs_path = os.path.abspath("data/small_data.root")
data_url = f"file://{data_abs_path}"

data_file = (data_url,)


events = NanoEventsFactory.from_root(
    {data_file: "/Events"}, metadata={"dataset": "SingleMu"}
).events()

q1_hist = (
    hda.Hist.new.Reg(100, 0, 200, name="met", label="$E_{T}^{miss}$ [GeV]")
    .Double()
    .fill(events.MET.pt)
)

q1_hist.compute(**executor_args).plot1d()

dak.necessary_columns(q1_hist)